# Sistem Rekomendasi Artikel Menggunakan Content-Based Filtering
# 1. Pendahuluan
Notebook ini berisi implementasi sistem rekomendasi artikel menggunakan pendekatan Content-Based Filtering.
Pendekatan ini bekerja dengan cara merekomendasikan artikel yang memiliki kemiripan konten dengan artikel yang telah dibaca pengguna.
Dataset diambil dari Kaggle: https://www.kaggle.com/datasets/jainilcoder/article-recommendation-system

# 2. Import Library
Tahap ini mengimpor seluruh pustaka yang dibutuhkan, seperti pandas untuk manipulasi data,
nltk untuk preprocessing teks, dan scikit-learn untuk vektorisasi serta penghitungan kemiripan antar dokumen.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re


## Download NLTK
Mendwonload korpus "stopwords" dari library NLTK

In [ ]:
nltk.download('stopwords')

# 3. Load Dataset
Memuat Dataset Artikel, Struktur dataset terdiri dari dua kolom utama:

1. `'Titles'`

2. `'Article'.`

In [ ]:
df = pd.read_csv('/content/articles.csv', encoding='latin1')
df.head()

# 4. Explatory Data Analysis (EDA)

## Melihat Informasi Data
 Menampilkan ringkasan informasi DataFrame, termasuk tipe data dan jumlah nilai non-null.

In [ ]:
print(df.info())

## Menangani Missing Values
Menghitung dan menampilkan jumlah nilai yang hilang (NaN) di setiap kolom DataFrame.

In [ ]:
print(df.isnull().sum())

## Melihat Sampel
Menampilkan 5 Sampel dari DataFrame

In [ ]:
print(df.sample(5))

# 5. Preprocessing
Tahapan ini bertujuan membersihkan teks dari karakter tidak penting,
menghapus stopwords, dan melakukan stemming untuk mendapatkan bentuk kata dasar.
Gabungan antara kolom judul dan isi artikel digunakan sebagai basis fitur teks.
"""

In [ ]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    text = str(text).lower()  # ubah ke huruf kecil
    text = re.sub(r'\d+', '', text)  # hapus angka
    text = text.translate(str.maketrans('', '', string.punctuation))  # hapus tanda baca
    tokens = text.split()
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]  # hapus stopwords dan stemming
    return ' '.join(tokens)

# Menggabungkan judul dan isi artikel
menggabungkan kolom `'Titles'` dan `'Article'` menjadi kolom `'text'`. Kemudian, membersihkan teks di kolom `'text'` menggunakan fungsi preprocess dan menyimpannya di kolom `'text_clean'`.

In [ ]:
df['text'] = df['Title'] + " " + df['Article']
df['text_clean'] = df['text'].apply(preprocess)

# 6. Feature Extraction
 Melakukan Ektraksi Fitur, TF-IDF (Term Frequency-Inverse Document Frequency) digunakan untuk mengubah teks menjadi vektor numerik
 berdasarkan frekuensi kemunculan kata dalam dokumen relatif terhadap seluruh korpus.

In [ ]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['text_clean'])

# 7. Similarity Calculation
Cosine similarity digunakan untuk mengukur tingkat kemiripan antar dua artikel berdasarkan vektor TF-IDF.

Nilainya berkisar antara 0 (tidak mirip) hingga 1 (sangat mirip).

In [ ]:
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 8. Rekomendasi Artikel
Fungsi get_recommendations menerima indeks artikel dan mengembalikan daftar top-n artikel paling mirip berdasarkan skor cosine similarity. Artikel input akan dilewati dalam hasil.

In [ ]:
def get_recommendations(index, top_n=10):
    sim_scores = list(enumerate(cos_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]  # Skip artikel itu sendiri
    article_indices = [i[0] for i in sim_scores]
    return df.iloc[article_indices][['Title']]

# Contoh rekomendasi
article_id = 10
print("Artikel utama:")
print(df.iloc[article_id]['Title'])
print("\nRekomendasi artikel:")
print(get_recommendations(article_id))


# 9. Evaluasi
Evaluasi dilakukan dengan cara observasi manual terhadap hasil rekomendasi artikel.

Contoh:

Best Books to Learn Computer Vision

Rekomendasi yang dihasilkan:
1. Best Books to Learn NLP
2. Best Books to Learn Deep Learning
3. Best Books to Learn Data Analysis
4. Best Python Frameworks to Build APIs
5. Best Resources to Learn Python
6. Voice Recorder using Python
7. Multiclass Classification Algorithms in Machine Learning
8. Applications of Deep Learning
9. Apple Stock Price Prediction with Machine Learning
10. Use Cases of Different Machine Learning Algorithms

Analisis:
- Mayoritas artikel yang direkomendasikan masih dalam topik yang berkaitan dengan *Machine Learning*, *Deep Learning*, dan *Python*, yang secara tematik memang berhubungan dengan Computer Vision.

- Judul-judul seperti “Best Books to Learn NLP” dan “Best Books to Learn Deep Learning” menunjukkan bahwa sistem berhasil mengenali pola dari artikel dengan format judul yang mirip dan konten serupa.

- Ini menunjukkan bahwa pendekatan Content-Based Filtering dengan TF-IDF dan cosine similarity bekerja cukup efektif pada dataset ini.

Keterbatasan:
- Karena tidak ada label relevansi atau interaksi pengguna, tidak bisa dihitung metrik seperti Precision@k atau Recall.

- Rekomendasi bersifat statis, belum mempertimbangkan preferensi atau histori pengguna.

Kesimpulan Evaluasi:

1. Rekomendasi yang diberikan cukup relevan dan kontekstual terhadap artikel sumber.

2. Evaluasi berbasis pengamatan ini cukup untuk memvalidasi pendekatan content-based filtering pada kasus ini.

# 10. Kesimpulan
Proyek ini berhasil membangun sistem rekomendasi artikel menggunakan pendekatan Content-Based Filtering
berdasarkan kemiripan konten antarartikel. Metode yang digunakan meliputi:

- Preprocessing teks: pembersihan teks, penghapusan stopwords, dan stemming.
- Ekstraksi fitur: TF-IDF Vectorizer digunakan untuk merepresentasikan artikel dalam bentuk vektor numerik.
- Penghitungan kemiripan: cosine similarity menghitung tingkat kemiripan antar artikel.

Hasil evaluasi menunjukkan bahwa sistem mampu menghasilkan rekomendasi artikel yang relevan secara tematik
terhadap artikel acuan. Sebagai contoh, untuk artikel berjudul "Best Books to Learn Computer Vision", sistem
berhasil merekomendasikan artikel lain yang membahas buku-buku pembelajaran terkait bidang NLP, Deep Learning,
dan Python, yang masih dalam satu ranah keilmuan.

Kelebihan:
- Tidak memerlukan data interaksi pengguna.

- Efektif pada kasus dengan teks deskriptif yang kaya informasi.

Keterbatasan:
- Rekomendasi bersifat statis dan belum mempertimbangkan preferensi pengguna secara personal.

- Tidak tersedia metrik kuantitatif untuk evaluasi (seperti precision@k), karena dataset tidak menyertakan label relevansi.

Rekomendasi Pengembangan:
- Menggabungkan pendekatan ini dengan metode collaborative filtering jika data pengguna tersedia.

- Menyediakan antarmuka input berdasarkan judul artikel untuk kenyamanan pengguna.

Dengan pendekatan content-based ini, sistem rekomendasi dapat digunakan sebagai solusi awal yang sederhana namun efektif
untuk menyajikan artikel serupa kepada pembaca berdasarkan isi kontennya.
